<a href="https://colab.research.google.com/github/venkatarajeshzilla/titanic-Passenger-Survival-Prediction-Using-Logistic-Regression/blob/main/Passenger_Survival_Prediction_Using_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/Titanic-Dataset.csv')

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [ ]:
df['Sex']=df['Sex'].map({'male':0,'female':1})

In [ ]:
x = df[['Pclass', 'Age', 'Sex', 'Fare']]  # make sure features are selected
   # encode 'Sex' if it's categorical

y=df['Survived']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
prediction=model.predict(x_test)

In [ ]:
accuracy=accuracy_score(y_test,prediction)

In [ ]:
print("Model acuuracy:",accuracy)

Model acuuracy: 0.7297297297297297


In [35]:
import pandas as pd

# Create a new test input (Pclass=1, Age=54, Sex=male(1), Fare=7.25)
new_input_df = pd.DataFrame([[1, 26, 0, 7.25]], columns=['Pclass', 'Age', 'Sex', 'Fare'])

# Predict survival
prediction = model.predict(new_input_df)

# Output human-readable result
if prediction[0] == 0:
    print('not survive')
else:
    print('survive')




survive


In [10]:
!pip install gradio -q

In [36]:
# First, ensure you have gradio installed in your Colab environment.
!pip install gradio -q

import pandas as pd
import gradio as gr

# ================================================================
#  --- YOUR MACHINE LEARNING MODEL GOES HERE ---
# ================================================================
# Load your actual trained model.
class YourTrainedModel:
    def predict(self, df):
        if df.iloc[0]['Pclass'] == 1 and df.iloc[0]['Age'] < 30:
            return [1]
        else:
            return [0]

model = YourTrainedModel()

# --- This is the prediction function the UI will call. ---
# This version returns both the prediction text and the raw prediction value.
def predict_survival(pclass, age, sex, fare):
    sex_value = 1 if sex == 'male' else 0
    new_input_df = pd.DataFrame([[pclass, age, sex_value, fare]],
                                 columns=['Pclass', 'Age', 'Sex', 'Fare'])
    prediction_value = model.predict(new_input_df)[0]

    if prediction_value == 0:
        prediction_text = 'Prediction: Not Survive'
    else:
        prediction_text = 'Prediction: Survive'

    return prediction_text, prediction_value

# --- This function handles the pop-up message based on the prediction. ---
def show_popup_message(prediction_value):
    # This is the corrected line: we convert the string back to an integer.
    if int(prediction_value) == 1:
        gr.Info("By God's grace he is survived")
    else:
        gr.Info("Sorry, unfortunately he did not survive")

# --- This function clears all the UI fields. ---
def clear_fields():
    return None, None, 'male', None, None, gr.update(visible=False), None

# --- Use a reliable, built-in Gradio theme with a simple color change. ---
custom_theme = gr.themes.Soft(primary_hue="orange")

# --- Define the Gradio UI using Blocks for a more structured layout. ---
with gr.Blocks(theme=custom_theme, title="Survival Prediction App") as demo:
    gr.Markdown(
        """
        # 🛳️ Titanic Survival Prediction

        This application predicts the survival outcome of a passenger on the Titanic using a machine learning model.
        Enter the passenger details below and press 'Predict' to see the result.

        ---
        """
    )

    with gr.Row():
        with gr.Column():
            pclass_input = gr.Number(label='Pclass (1-3)', minimum=1, maximum=3, precision=0)
            age_input = gr.Number(label='Age', minimum=0, maximum=100)
            sex_input = gr.Radio(['male', 'female'], value='male', label='Sex')
            fare_input = gr.Number(label='Fare', minimum=0)

            with gr.Row():
                predict_button = gr.Button("Predict Survival", variant="primary")
                clear_button = gr.Button("Clear", variant="secondary")

        with gr.Column():
            output_text = gr.Text(label="Prediction Result", visible=False)
            raw_prediction_value = gr.Textbox(visible=False)

    # Connect the "Predict" button to the prediction function.
    predict_button.click(
        fn=predict_survival,
        inputs=[pclass_input, age_input, sex_input, fare_input],
        outputs=[output_text, raw_prediction_value]
    ).then(
        fn=lambda: gr.update(visible=True),
        outputs=output_text
    ).then(
        fn=show_popup_message,
        inputs=raw_prediction_value,
        outputs=None
    )

    # Connect the "Clear" button to the function that clears all fields.
    clear_button.click(
        fn=clear_fields,
        inputs=None,
        outputs=[pclass_input, age_input, sex_input, fare_input, output_text, raw_prediction_value]
    )

    gr.Markdown(
        """
        ---
        **Note:** This is a demonstration. Replace the placeholder model with your own.

        Made by [Your Name] | [Link to your GitHub profile]
        """
    )

# Launch the final, polished UI.
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b20399105e00c85946.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
